In [1]:
import mlflow
from mlflow.models import infer_signature # To normalize input and output schema
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### 1. Loading dataset and training a simple model

In [2]:
# Load the Iris dataset.
X, y = load_iris(return_X_y=True)

# Split the dataset into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definining params.
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    # "multi_class": "multinomial",
    "random_state": 42,
}

In [3]:
# Train the model
lr = LogisticRegression(**params)   
lr.fit(X_train, y_train)

# Make predictions
y_pred = lr.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Accuracy:", acc)

Accuracy: 1.0


### 2. Log the model and its metadata to MLflow

In [4]:
# Tracking server uri
mlflow.set_tracking_uri("http://localhost:5050")

# Create a new MLflow experiment
mlflow.set_experiment("Iris Classification")

# Start an MLflow run
with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params(params)
    
    # Log the loss metric
    mlflow.log_metric("accuracy", acc)
    
    # Model signature
    signature = infer_signature(X_train, lr.predict(X_train))
    
    # Log the model, inheriting the params and metrics
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        name="iris_model",
        signature=signature,
        input_example=X_train,
        # registered_model_name="IrisClassificationModel"
    )
    
    # Set a tag for the run
    mlflow.set_logged_model_tags(
        model_info.model_id, {"version": "v1.0", "author": "fenilenvinileno97"}
    )

2025/09/13 14:43:21 INFO mlflow.tracking.fluent: Experiment with name 'Iris Classification' does not exist. Creating a new experiment.


🏃 View run incongruous-foal-262 at: http://localhost:5050/#/experiments/150557764733201083/runs/6cea25a2c6b745d2a7b21acc23ca2ce3
🧪 View experiment at: http://localhost:5050/#/experiments/150557764733201083
